In [58]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [59]:
df = pd.read_csv("../input/product-position/upper.csv", header=None)
df.head()

In [60]:
df.shape

In [61]:
df.groupby(0)[0].count()

In [62]:
df_sel = df[(df[0]==0) | (df[0]==8) | (df[0]==9)]
df_sel.shape

In [63]:
Xo = df_sel.drop([0],axis=1).values
yo = df_sel[0].values

In [64]:
Xo.shape

In [65]:
yo.shape

In [66]:
np.unique(yo)

In [67]:
yo[yo==8] = 1
yo[yo==9] = 2
np.unique(yo)

In [68]:
[sum(yo == item) for item in np.unique(yo)]

In [69]:
from sklearn.model_selection import train_test_split
X_tr,X_te,y_tr,y_te = train_test_split(Xo,yo,test_size=0.2,random_state=0)

In [70]:
X_tr, X_val, y_tr, y_val = train_test_split(X_tr, y_tr, test_size=0.2, random_state=1)

In [71]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1)) 
scaler.fit(X_tr)
X_train_norm = scaler.transform(X_tr)
X_val_norm = scaler.transform(X_val)
X_test_norm = scaler.transform(X_te)

# Logistic

In [72]:
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import GridSearchCV 

params = {'C': [1, 2, 4, 8, 16, 32]} 
clf = GridSearchCV(LogisticRegression(random_state=0, solver='liblinear'),params, cv=10)
clf.fit(X_train_norm, y_tr)
print("Best params : " + str(clf.best_params_))
print("10CV accuracy : "+str(clf.best_score_*100))

In [73]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
y_predict = clf.predict(X_test_norm) #ทำนาย
print("Test accuracy : "+str(sum(y_te == y_predict)/len(y_te)*100))

In [74]:
!pip install tensorflow==2.0.0-beta1
import tensorflow as tf

In [87]:
tf.compat.v1.reset_default_graph() # Clear Model
np.random.seed(1) # Fix Randomness
tf.compat.v1.random.set_random_seed(1) # Fix Randomness
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(8, activation='relu', input_shape=(1174,)),#colum
  tf.keras.layers.Dense(4, activation='relu'),
  tf.keras.layers.Dense(2, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid') # y=0,1 sigmoid
])
model.summary()

In [88]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [89]:
#Train Model
train_acc = list()
val_acc = list()
for i in range(0,500):
  history = model.fit(X_train_norm, y_tr, epochs= 1, batch_size = 200, validation_data= (X_val_norm, y_val))
  tmp_avg = np.mean(history.history['accuracy'])
  tmp_avg_val = np.mean(history.history['val_accuracy'])
  train_acc.append(tmp_avg)
  val_acc.append(tmp_avg_val)

In [90]:
history.history

In [91]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
plt.figure(num=None, figsize=(16, 8), dpi=90, facecolor='w', edgecolor='k')
plt.plot()
plt.plot(train_acc)
plt.plot(val_acc)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [92]:
#Test Accuracy
y_predict = np.round(model.predict(X_test_norm))
y_predict = [i[0] for i in y_predict.tolist()]
sum(y_predict == y_te)/len(y_te)

**Feature Selection**

In [94]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
fs = SelectKBest(f_classif, k=5)
fs.fit(X_tr, y_tr)
X_train_fs = fs.transform(X_tr)
X_test_fs = fs.transform(X_te)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train_fs)
X_train_norm = scaler.transform(X_train_fs)
X_test_norm = scaler.transform(X_test_fs)

In [104]:
clf = LogisticRegression(random_state=0, solver='liblinear')
acc = cross_val_score(clf,X_train_norm,y_tr, cv=10)
print("10CV Training Accuracy: "+str(acc.mean()))

In [103]:
clf = LogisticRegression(random_state=0, solver='liblinear')
clf.fit(X_train_norm, y_tr)
yp = clf.predict(X_test_norm)
acc = sum(yp == y_te)/len(y_te)
print("Test Training accuracy : "+str(acc))

PCA

In [108]:
from sklearn.decomposition import PCA  
pca = PCA(n_components = 5) 
pca.fit(X_tr)  
X_train_pca = pca.transform(X_tr)
X_test_pca =  pca.transform(X_te)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1)) 
scaler.fit(X_train_pca)
X_train_norm = scaler.transform(X_train_pca)
X_test_norm = scaler.transform(X_test_pca)
clf = LogisticRegression(random_state=0, solver='liblinear')
acc = cross_val_score(clf,X_train_norm,y_tr, cv=10)
print("10CV Training Accuracy: "+str(acc.mean()))

In [109]:
clf = LogisticRegression(random_state=0, solver='liblinear')
clf.fit(X_train_norm, y_tr)
yp = clf.predict(X_test_norm)
acc = sum(yp == y_te)/len(y_te)
print("Test Training accuracy : "+str(acc))

In [110]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV 

params = {'C': [1, 2, 4, 8, 16, 32]} 
clf = GridSearchCV(LogisticRegression(random_state=0, solver='liblinear'),params, cv=10)
clf.fit(X_train_norm, y_tr)
print("Best params : " + str(clf.best_params_))
print("10CV accuracy : "+str(clf.best_score_*100))


In [111]:
clf = LogisticRegression(random_state=0, solver='liblinear')
clf.fit(X_train_norm, y_tr)
yp = clf.predict(X_test_norm)
acc = sum(yp == y_te)/len(y_te)
print("Test Training accuracy : "+str(acc))

# ExtraTreesClassifier +smote

In [128]:
from sklearn.preprocessing import MinMaxScaler 
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import Pipeline 

from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=0)
X_train, y_train = sm.fit_resample(X_train, y_train)

clf = Pipeline([
  ('scaler', MinMaxScaler(feature_range=(0, 1))), 
  ('classification', ExtraTreesClassifier(random_state=0))
])

In [129]:
from sklearn.model_selection import GridSearchCV

params = {
    'classification__n_estimators': [10, 30, 50, 70, 90]
}
top_clf = GridSearchCV(clf,params, cv=10)
top_clf.fit(X_train, y_train)

In [130]:
top_clf.best_score_

In [131]:
y_predict = top_clf.predict(X_test)
acc = sum(y_predict == y_test)/len(y_test)
print("Test accuracy : "+str(acc))